In [14]:
from pathlib import Path
from datetime import datetime, timedelta

def average_latency(sub_log: Path) -> float:
	readings = []
	with sub_log.open() as f:
		for line in f:
			if " - " in line:
				ts, val = line.split(" - ")
				readings.append((datetime.fromisoformat(ts), datetime.fromisoformat(val.strip())))

	# remove last value (is unusally large latency due to timeout)
	readings = readings[:-2]

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	# add 1s to values (cache flushing issue)
	readings = [(ts, val + timedelta(seconds=1))  for ts, val, in readings]


	latency = [(ts-val).total_seconds() for ts, val in readings]

	return sum(latency)/len(latency)

def average_cpu_usage(relay_perf: Path) -> float:
	readings = []
	with relay_perf.open() as f:
		for line in f:
			ts, val = line.split("   ")
			if val == "":
				raise ValueError("empty val")
			readings.append((datetime.fromisoformat(ts.strip()[:-1]), float(val.strip())))

	# only keep last 50 seconds to be safe
	last_ts = readings[-2][0]
	readings = [(ts, val) for ts, val in readings if ts > last_ts - timedelta(seconds=50)]

	percentages = [v for _, v in readings]

	return sum(percentages)/len(percentages)


def load_measurements(dir: Path):
	measurements = dir / Path("measurements")
	if not measurements.is_dir():
		raise ValueError(f"invalid directory: {measurements}")

	sub_logs = []
	relay_perf = []

	# organize files
	for file in measurements.iterdir():
		if file.name.startswith("clock"):
			sub_logs.append(file)
		if file.name.endswith("cpu_usage.log"):
			relay_perf.append(file)

	sub_avg_latencies = []
	for sub in sub_logs:
		try:
			sub_avg_latencies.append(average_latency(sub))
		except Exception as e:
			pass#print(f"{sub} skipped due to {e}")

	relay_perf_avg = {}
	for relay in relay_perf:
		name = relay.stem.replace("_cpu_usage", "")
		try:
			relay_perf_avg[name] = average_cpu_usage(relay)
		except Exception as e:
			print(f"{name} skipped due to {e}")


	return sub_avg_latencies, relay_perf_avg

load_measurements(Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement/star"))

([0.016365461538461534,
  0.016150807692307693,
  0.016522961538461542,
  0.016469313725490195,
  0.01351019607843137,
  0.013224346153846149,
  0.012233274509803924,
  0.012381254901960778,
  0.012391647058823528,
  0.012550882352941176,
  0.016488764705882356,
  0.017215942307692304,
  0.012965078431372552,
  0.009082529411764706,
  0.015936941176470588,
  0.008633519230769231,
  0.009133673076923077,
  0.008616557692307692,
  0.012907057692307692,
  0.01571227450980392,
  0.012356019607843138,
  0.015996269230769234,
  0.01656627450980392,
  0.01664650980392157,
  0.015919921568627454,
  0.01675621153846154,
  0.013602352941176468,
  0.015377692307692311,
  0.01546390196078431,
  0.012281450980392156,
  0.008468274509803924,
  0.016662666666666662,
  0.009690326923076923,
  0.013390326923076925,
  0.016433039215686274,
  0.015235960784313728,
  0.01647882352941177,
  0.008803461538461538,
  0.015664941176470586,
  0.016497461538461537,
  0.01651445098039215,
  0.012639137254901962,


In [15]:
from pprint import pprint
base = Path("/Users/earvarm/Documents/repos/moq-rs/dev/performance-measurement")

measurements = []
for topo in ["full-mesh", "star", "leaf-and-spine", "two-relays", "single-relay"]:
	latencies, relays = load_measurements(base / topo)

	avg_lat = sum(latencies) / len(latencies)
	topo_results = {
		"name": topo,
		"average_latency": avg_lat,
		"relay_utilizations": relays
	}
	measurements.append(topo_results)
	pprint(topo_results)

{'average_latency': 0.011043172023001506,
 'name': 'full-mesh',
 'relay_utilizations': {'relay1': 0.23061224489795917,
                        'relay2': 0.2285714285714285,
                        'relay3': 0.25102040816326526,
                        'relay4': 0.24285714285714283,
                        'relay5': 0.24897959183673476}}
{'average_latency': 0.013657955092006032,
 'name': 'star',
 'relay_utilizations': {'hub': 0.42244897959183686,
                        'relay1': 0.33469387755102037,
                        'relay2': 0.3081632653061224,
                        'relay3': 0.3061224489795918,
                        'relay4': 0.30816326530612237}}
{'average_latency': 0.010920719680241327,
 'name': 'leaf-and-spine',
 'relay_utilizations': {'leaf1': 0.48999999999999977,
                        'leaf2': 0.48200000000000004,
                        'leaf3': 0.45599999999999985,
                        'spine1': 0.48200000000000004,
                        'spine2': 0.507999999

In [16]:
# scale CPU utilization to highest
highest_cpu = max(max(t["relay_utilizations"].values()) for t in measurements)

for t in measurements:
	t["average_latency"] = f"{t['average_latency'] * 1000} ms"
	for r, val in t["relay_utilizations"].items():
		t["relay_utilizations"][r] = f"{val / highest_cpu:.3%}"
pprint(measurements)

[{'average_latency': '11.043172023001505 ms',
  'name': 'full-mesh',
  'relay_utilizations': {'relay1': '45.396%',
                         'relay2': '44.994%',
                         'relay3': '49.413%',
                         'relay4': '47.807%',
                         'relay5': '49.012%'}},
 {'average_latency': '13.657955092006032 ms',
  'name': 'star',
  'relay_utilizations': {'hub': '83.159%',
                         'relay1': '65.885%',
                         'relay2': '60.662%',
                         'relay3': '60.260%',
                         'relay4': '60.662%'}},
 {'average_latency': '10.920719680241328 ms',
  'name': 'leaf-and-spine',
  'relay_utilizations': {'leaf1': '96.457%',
                         'leaf2': '94.882%',
                         'leaf3': '89.764%',
                         'spine1': '94.882%',
                         'spine2': '100.000%'}},
 {'average_latency': '9.058794104685145 ms',
  'name': 'two-relays',
  'relay_utilizations': {'relay1'